# Cities With Completed Trades  
**Robinhood SQL Interview Question**

---

## Question

Assume you're given the tables containing completed trade orders and user details in a Robinhood trading system.

Write a query to retrieve the **top three cities** that have the **highest number of completed trade orders**, listed in **descending order**. Output the city name and the corresponding number of completed trade orders.

---

## Schema

### `trades` Table:
| Column Name | Type                      |
|-------------|---------------------------|
| order_id    | integer                   |
| user_id     | integer                   |
| quantity    | integer                   |
| status      | string ('Completed', 'Cancelled') |
| date        | timestamp                 |
| price       | decimal (5, 2)            |

#### Example Input:
| order_id | user_id | quantity | status    | date                | price |
|----------|---------|----------|-----------|---------------------|-------|
| 100101   | 111     | 10       | Cancelled | 08/17/2022 12:00:00 | 9.80  |
| 100102   | 111     | 10       | Completed | 08/17/2022 12:00:00 | 10.00 |
| 100259   | 148     | 35       | Completed | 08/25/2022 12:00:00 | 5.10  |
| 100264   | 148     | 40       | Completed | 08/26/2022 12:00:00 | 4.80  |
| 100305   | 300     | 15       | Completed | 09/05/2022 12:00:00 | 10.00 |
| 100400   | 178     | 32       | Completed | 09/17/2022 12:00:00 | 12.00 |
| 100565   | 265     | 2        | Completed | 09/27/2022 12:00:00 | 8.70  |

---

### `users` Table:
| Column Name | Type     |
|-------------|----------|
| user_id     | integer  |
| city        | string   |
| email       | string   |
| signup_date | datetime |

#### Example Input:
| user_id | city          | email                           | signup_date          |
|---------|---------------|----------------------------------|----------------------|
| 111     | San Francisco | rrok10@gmail.com                 | 08/03/2021 12:00:00  |
| 148     | Boston        | sailor9820@gmail.com             | 08/20/2021 12:00:00  |
| 178     | San Francisco | harrypotterfan182@gmail.com      | 01/05/2022 12:00:00  |
| 265     | Denver        | shadower_@hotmail.com            | 02/26/2022 12:00:00  |
| 300     | San Francisco | houstoncowboy1122@hotmail.com    | 06/30/2022 12:00:00  |

---

## Example Output:
| city           | total_orders |
|----------------|--------------|
| San Francisco  | 3            |
| Boston         | 2            |
| Denver         | 1            |

In the given dataset, San Francisco has the highest number of completed trade orders with 3 orders. Boston holds the second position with 2 orders, and Denver ranks third with 1 order.

---


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType, DoubleType
from datetime import datetime
from pyspark.sql.functions import *

# Start Spark session
spark = SparkSession.builder.master('local[1]').appName("RobinhoodTrades").getOrCreate()

# Define schema for trades
trades_schema = StructType([
    StructField("order_id", IntegerType(), True),
    StructField("user_id", IntegerType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("status", StringType(), True),
    StructField("date", TimestampType(), True),
    StructField("price", DoubleType(), True)
])

# Sample data for trades
trades_data = [
    (100101, 111, 10, "Cancelled", datetime(2022, 8, 17, 12, 0, 0), 9.80),
    (100102, 111, 10, "Completed", datetime(2022, 8, 17, 12, 0, 0), 10.00),
    (100259, 148, 35, "Completed", datetime(2022, 8, 25, 12, 0, 0), 5.10),
    (100264, 148, 40, "Completed", datetime(2022, 8, 26, 12, 0, 0), 4.80),
    (100305, 300, 15, "Completed", datetime(2022, 9, 5, 12, 0, 0), 10.00),
    (100400, 178, 32, "Completed", datetime(2022, 9, 17, 12, 0, 0), 12.00),
    (100565, 265, 2, "Completed", datetime(2022, 9, 27, 12, 0, 0), 8.70)
]

# Create trades DataFrame
trades_df = spark.createDataFrame(trades_data, schema=trades_schema)

# Define schema for users
users_schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("city", StringType(), True),
    StructField("email", StringType(), True),
    StructField("signup_date", TimestampType(), True)
])

# Sample data for users
users_data = [
    (111, "San Francisco", "rrok10@gmail.com", datetime(2021, 8, 3, 12, 0, 0)),
    (148, "Boston", "sailor9820@gmail.com", datetime(2021, 8, 20, 12, 0, 0)),
    (178, "San Francisco", "harrypotterfan182@gmail.com", datetime(2022, 1, 5, 12, 0, 0)),
    (265, "Denver", "shadower_@hotmail.com", datetime(2022, 2, 26, 12, 0, 0)),
    (300, "San Francisco", "houstoncowboy1122@hotmail.com", datetime(2022, 6, 30, 12, 0, 0))
]

# Create users DataFrame
users_df = spark.createDataFrame(users_data, schema=users_schema)

# Show data
trades_df.show()
users_df.show()


+--------+-------+--------+---------+-------------------+-----+
|order_id|user_id|quantity|   status|               date|price|
+--------+-------+--------+---------+-------------------+-----+
|  100101|    111|      10|Cancelled|2022-08-17 12:00:00|  9.8|
|  100102|    111|      10|Completed|2022-08-17 12:00:00| 10.0|
|  100259|    148|      35|Completed|2022-08-25 12:00:00|  5.1|
|  100264|    148|      40|Completed|2022-08-26 12:00:00|  4.8|
|  100305|    300|      15|Completed|2022-09-05 12:00:00| 10.0|
|  100400|    178|      32|Completed|2022-09-17 12:00:00| 12.0|
|  100565|    265|       2|Completed|2022-09-27 12:00:00|  8.7|
+--------+-------+--------+---------+-------------------+-----+

+-------+-------------+--------------------+-------------------+
|user_id|         city|               email|        signup_date|
+-------+-------------+--------------------+-------------------+
|    111|San Francisco|    rrok10@gmail.com|2021-08-03 12:00:00|
|    148|       Boston|sailor9820@g

In [5]:
trades_df.join(users_df,['user_id'],'inner')\
    .where('status="Completed"')\
    .groupBy('city').agg(count('order_id').alias('total_orders'))\
    .orderBy('total_orders',ascending=0).limit(3).show()

+-------------+------------+
|         city|total_orders|
+-------------+------------+
|San Francisco|           3|
|       Boston|           2|
|       Denver|           1|
+-------------+------------+



In [10]:
trades_df.createOrReplaceTempView('trades')
users_df.createOrReplaceTempView('users')

spark.sql(
'''
select city,count(order_id) as total_orders
from 
trades join users using(user_id)
where(status="Completed")
group by city 
order by total_orders desc
limit 3
'''
).show()

+-------------+------------+
|         city|total_orders|
+-------------+------------+
|San Francisco|           3|
|       Boston|           2|
|       Denver|           1|
+-------------+------------+

